# Summary analysis of Ablation experiments
## 81c NTR experiment b
## 1026 NTR-UTR experiment
## DT laser ablation


In [ ]:
import socket
import os
import pandas as pd

In [ ]:
if 'startDirMaster' not in locals():
    startDirMaster=os.getcwd()

propsFn=startDirMaster+'\\props.csv'

props=pd.read_csv(propsFn, header=None, index_col=0, squeeze=True,delim_whitespace=True).to_dict()
#print(props)
props

In [ ]:
codeDir = '..\\jlsocialbehavior'
    

if 'NB_path' not in locals():
    NB_path = os.getcwd()
else:
    os.chdir(NB_path)
    

base=props['BaseDir']
#artDir=props['ArtDir']
RawDataDir = os.path.join(base,props['RawDataDir'])+'\\'+'behavior\\'
ProcessingDir = os.path.join(base,props['ProcessingDir'])+'\\'
outputDir = os.path.join(base,props['outputDir'])+'\\'
FigureDir = os.path.join(base,props['FigureDir'])+'\\'

expFile=props['allExpBehFn']
anFile=props['allAnBehFn']


props

print('baseFolder = ' + base)



os.chdir(codeDir)
props

In [ ]:
#-------- experiment variables

tmin=80 # initial experiment time to exclude from analysis
tmax=240 # time limit for analysis
vmax=12 # upper limit of average swim speed to include animals. 
vmin=1.5
bdmax=1.5 #upper limit of average bout duration to include animals.

In [ ]:
%config InteractiveShellApp.pylab_import_all = False
%matplotlib inline
%pylab inline
%reload_ext autoreload
%autoreload 2

import sys
import fnmatch
import numpy as np
import math
import matplotlib.pyplot as plt
import pandas as pd
from pandas import DataFrame, Series
import seaborn as sns
import glob
from datetime import datetime
from scipy import stats
import scipy.stats as sta
import scipy.optimize
import statsmodels.stats.api as sms

import functions.matrixUtilities_joh as mu
import functions.notebookHelper as nh
import matplotlib.pyplot as plt
import models.experiment as xp
import models.experiment_set as es
import functions.paperFigureProps as pfp
import functions.peakdet as pkd

pfp.paper()
inToCm=2.54
import matplotlib as mpl
mpl.rcParams['svg.fonttype'] = 'none'

In [ ]:
info=pd.read_excel(expFile)
info=info[info.set.isin(['d','e','f'])]
info

In [ ]:
infoAn=pd.read_excel(anFile)
infoAn.tail()

In [ ]:
# collect meta information and save to new csv file for batch processing

aviPath=[]
posPath=[]
PLPath=[]
expTime = []
birthDayAll=[]
anIDsAll=[]
camHeightAll=[]
arenaDiamAll=[]
anPdishAll=[]


#--------------
camHeight=[105,180] #for arena up,dn ### This won't work on these Videos with 25mm lens!!!
#--------------

for index,row in info.iterrows():

    startDir=RawDataDir+row.path+'\\'
    if not os.path.isdir(startDir):
        startDir=RawAlt+row.path+'\\'
    if not os.path.isdir(startDir):
        startDir=RawAlt2+row.path+'\\'
        #print(index,startDir)
    
    posPath.append(glob.glob(startDir+'PositionTxt*.txt')[0])
    PLPath.append(glob.glob(startDir+'PL*')[0])
    
    head, tail = os.path.split(posPath[-1])
    currTime=datetime.strptime(tail[-23:-4], '%Y-%m-%dT%H_%M_%S')
    expTime.append(currTime)
    
    camHeightAll.append(camHeight[('_dn_' in head)*1])
    
    anNrs=row.anNr #Note that anNrs are 1 based!
    if ':' in anNrs:
        a,b=anNrs.split(sep=':')
        anNrs=np.arange(int(a),int(b)+1)
    else:
        anNrs=np.array(anNrs.split()).astype(int)
        
    anIDs=anNrs #-1 no more 0-based since using pandas merge to find animal numbers
    anIDsAll.extend(anIDs)

    bd=infoAn[infoAn.anNr.isin(anIDs)].bd.values
    #bd=infoAn.bd.values[anIDs-1] #a bit dirty to use anIDs directly here. Should merge
    birthDayAll.append(' '.join(list(bd)))
    
    arenaDiamAll.append(row.arenaDiam)
    anPdishAll.append(row.anPdish)

info['camHeight']=camHeightAll
info['txtPath']=posPath
info['pairList']=PLPath
info['aviPath']='default'
info['birthDayAll']=birthDayAll
info['epiDur'] = 5      # duration of individual episodes (default: 5 minutes)
info['episodes'] = 54   # number of episodes to process: -1 to load all episodes (default: -1)
info['inDish'] = 10#np.arange(len(posPath))*120     # time in dish before experiments started (default: 10)
info['arenaDiameter_mm'] = arenaDiamAll # arena diameter (default: 100 mm)
info['anPdish']=anPdishAll
info['minShift'] = 60 # minimum number of seconds to shift for control IAD
info['episodePLcode'] = 0 # flag if first two characters of episode name encode animal pair matrix (default: 0)
info['recomputeAnimalSize'] = 0 # flag to compute animals size from avi file (takes time, default: 1)
info['SaveNeighborhoodMaps'] = 1 # flag to save neighborhood maps for subsequent analysis (takes time, default: 1)
info['computeLeadership'] = 0 # flag to compute leadership index (takes time, default: 1)
info['ComputeSync'] = 0 # flag to compute swim synchronization with stimulus (takes time, default: 1)
info['ComputeBouts'] = 1 # flag to compute swim bout frequency (takes time, default: 1)
#info['set'] = np.arange(len(posPath))   # experiment set: can label groups of experiments (default: 0)
info['ProcessingDir']=ProcessingDir
info['outputDir']=outputDir
info['expTime']=expTime

In [ ]:
startDir

In [ ]:
posPath

In [ ]:
csvFile=os.path.join(ProcessingDir,'processingSettings.csv')
info.to_csv(csvFile,encoding='utf-8')
info.tail()

In [ ]:
rereadData=1
if rereadData:
    def readExperiment(keepData=True):
        tmp=es.experiment_set(csvFile=csvFile,MissingOnly=True)
        if keepData:
            return tmp
        else:
            return 1

    expSet=readExperiment(keepData=False)

In [ ]:
csvPath = []
for f in [mu.splitall(x)[-1][:-4] for x in info.txtPath]:
    csvPath.append(glob.glob(ProcessingDir+f+'*siSummary*.csv')[0])

df=pd.DataFrame()

startAn=0

for i,fn in enumerate(csvPath):
    
    tmp=pd.read_csv(fn,index_col=0,sep=',')
    tmp['experiment']=tmp.animalSet
    tmp.animalSet=i
    
    nrAnimalsPerDish=info.anPdish.values[i]
    print(fn,nrAnimalsPerDish,' animals per setup')
    tmp.animalIndex=tmp.animalIndex+startAn
    startAn=startAn+nrAnimalsPerDish
    tmp['anIx']=tmp.animalIndex
    tmp.animalIndex=np.array(anIDsAll)[tmp.animalIndex]
    df=pd.concat([df,tmp])

df['episode']=[x.strip().replace('_','') for x in df['episode']]
df=pd.merge(df,infoAn[['anNr','line','group']],left_on='animalIndex',right_on='anNr',how='left')
df=pd.merge(df,info[['date']],left_on='animalSet',right_on=info.index,how='left')

print('df shape',df.shape)
df.tail(10)

## Habituation or Fatigue within 20 hours?

In [ ]:
def sem(x):
    return np.std(x,axis=0)/np.sqrt(x.shape)

x=np.random.random(100)

def ci95(x):
    return np.nanmean(x)-sms.DescrStatsW(x[np.isfinite(x)]).tconfint_mean()[0]

print('std of uniform = 0.2886751345948129. STDdata:',np.std(x), 'semData:',sem(x),'samples:',x.shape) 
print('ci95:',ci95(x))


Plot shoaling index during closed loop skype episodes over time.

In [ ]:
dfPlot=(df[df.group!='e'].groupby(['inDishTime','episode']).si.agg(['mean','std',sem,ci95])
    .unstack()
    .stack(dropna=True)
    .reset_index())

dfPlot.head()


In [ ]:
fig, axes = plt.subplots(figsize=(6, 3))
g=sns.scatterplot(data=dfPlot,x='inDishTime',hue='episode',y='mean')
plt.xlim([0,5.5*60])
plt.ylim([0,.30])
plt.xlabel('Time (Minutes)')
plt.ylabel('Attraction')
plt.title('Mean attraction, all animals')
plt.axvline(tmin)
plt.axvline(tmax)

new_title = 'Stimulus dot motion'
g.legend_.set_title(new_title)
# replace labels
#plt.legend(ncol=1,handletextpad=0,bbox_to_anchor=(1, 1),loc='upper left')

new_labels = ['Bout','Smooth']
for t, l in zip(g.legend_.texts, new_labels): t.set_text(l)
sns.despine()


# Mean response over all stimuli per animal

In [ ]:
df.info()

In [ ]:
sns.set_palette('viridis',3)
co=sns.color_palette("viridis", 3)
idx=(df['inDishTime']<tmax) & (df['inDishTime']>tmin)&(df.group!='e')
dfDR=df[idx]
dfEpiAn=dfDR.groupby(['episode','animalIndex','line','group','experiment'],sort=True).mean().reset_index()


In [ ]:
sns.pairplot(data=dfEpiAn[(dfEpiAn.experiment=='e')&(dfEpiAn.episode=='02k20f')&(dfEpiAn.boutDur<3)]
             ,vars=['si','avgSpeed_smooth','boutDur','thigmoIndex'],hue='group')

In [ ]:
showepis=['10k20f','02k20f']

In [ ]:
sns.pairplot(data=dfEpiAn[(dfEpiAn.episode.isin(showepis))],
             hue='group',
             markers="+",
             height=1.5,
             vars=['si','avgSpeed_smooth','thigmoIndex','boutDur']);

## there are some outlier animals:
###  with high speed >10, or with high boutDur >1.2

In [ ]:
showepis

In [ ]:
dfEpiAn[(dfEpiAn.experiment=='f')&dfEpiAn.episode.isin(showepis)]

In [ ]:


ix=(dfEpiAn.episode.isin(showepis))&((dfEpiAn.avgSpeed_smooth<vmin)|(dfEpiAn.avgSpeed_smooth>vmax)|(dfEpiAn.boutDur>bdmax))


excludeAnimals=dfEpiAn[ix].animalIndex.values
print('excluding animals: ',excludeAnimals)
print('vmax: ',vmax)
print('vmin: ',vmin)
print('bdmax: ',bdmax)

print('tmin: ',tmin)
print('tmax: ',tmax)

In [ ]:
sns.pairplot(data=dfEpiAn[(dfEpiAn.experiment=='e')&(dfEpiAn.episode=='02k20f')&(dfEpiAn.boutDur<3)]
             ,vars=['si','avgSpeed_smooth','boutDur','thigmoIndex'],hue='group')

In [ ]:
df.animalIndex.unique()

In [ ]:
idxExclude=(df['inDishTime']<tmax) & (df['inDishTime']>tmin)&(df.group!='e')&(~df.animalIndex.isin(excludeAnimals))
dfDR=df[idxExclude]
dfEpiAn=dfDR.groupby(['episode','animalIndex','line','group','experiment'],sort=True).mean().reset_index()

In [ ]:
sns.pairplot(data=dfEpiAn[(dfEpiAn.experiment=='e')&(dfEpiAn.episode=='02k20f')&(dfEpiAn.boutDur<3)]
             ,vars=['si','avgSpeed_smooth','boutDur','thigmoIndex'],hue='group')

In [ ]:
sns.pairplot(data=dfEpiAn[(dfEpiAn.episode.isin(showepis))],
             hue='group',
             markers="+",
             height=1.5,
             vars=['si','avgSpeed_smooth','thigmoIndex','boutDur']);

In [ ]:
from matplotlib.lines import Line2D
def cohen_d(x,y):
    nx = len(x)
    ny = len(y)
    dof = nx + ny - 2
    return (np.mean(x) - np.mean(y)) / np.sqrt(((nx-1)*np.std(x, ddof=1) ** 2 + (ny-1)*np.std(y, ddof=1) ** 2) / dof)

def plotGroupComparison(data,groups,param,labels,ylims=[],legendOffset=2):
    ncols=np.array(param).shape[0]
    fig, ax = plt.subplots(1,ncols,figsize=(ncols*3/inToCm,5/inToCm))
    
    palSimpleBar=['gray','r']
    
    for i,p in enumerate(param):
        sns.pointplot(y=p,
                    x='group',
                    hue='group',
                      order=groups,
                      hue_order=groups,
                    data=data,
                    ax=ax[i],
                      join=False,
                      color='k',
                      legend=True,
                    #palette=palSimpleBar,
                    linewidth=2,
                    ci='sd',
                     zorder=100)

        sns.stripplot(y=p, 
                    x='group',
                      hue='group',
                      order=groups,
                      hue_order=groups,
                    data=data,
                    ax=ax[i],
                    palette=palSimpleBar,
                      alpha=.5,
                      clip_on=False,
                      zorder=-100
                    #linewidth=2,
                    )

        ax[i].set_xlabel('')
        ax[i].set_xticks([])
        ax[i].set_ylabel(labels[i])
        if ylims:
            ax[i].set_ylim(ylims[i])

        sns.despine()
        ax[i].legend().remove()
        
        d2 = data.loc[data['group'] == groups[0],p]#iloc[:,2:]
        d1 = data.loc[data['group'] == groups[1],p]#iloc[:,2:]


        s,p=sta.ttest_ind(d1[~np.isnan(d1)],d2[~np.isnan(d2)])
        
        d=cohen_d(d1[~np.isnan(d1)],d2[~np.isnan(d2)])

        
        if (i>-10):
            ax[i].set_ylim(bottom=0,top=None)
        else:
            ax[i].axhline(0,ls=":",color='gray')
            
        txtHeight=ax[i].get_ylim()[1]+(np.diff(ax[i].get_ylim())*0.06)
        a = "p={:01.3f}".format(p) if p >= 0.001 else "p={:01.0e}".format(p)
        if p<0.05:
            txtCol='k'
        else:
            txtCol='gray'
        ax[i].text(-0.4,txtHeight,a,color=txtCol)
        txtHeight=ax[i].get_ylim()[1]+(np.diff(ax[i].get_ylim())*0.16)
        ax[i].text(-0.4,txtHeight,"d={:01.2f}".format(d),color=txtCol)
        
        
        
    plt.tight_layout()
    
    # Create dummy Line2D objects for legend
    h1 = Line2D([0], [0], marker='o', markersize=np.sqrt(20), color=palSimpleBar[0], linestyle='None')
    h2 = Line2D([0], [0], marker='o', markersize=np.sqrt(20), color=palSimpleBar[1], linestyle='None')

    
    #l=ax[i].legend()
    #h,l = ax[i].get_legend_handles_labels()
    
    ax[i].legend([h1, h2], ['Ctr','Abl'],frameon=False,ncol=1,
    #ax[i].legend(h[2:], ['Ctr','Abl'],frameon=False,ncol=1,
    loc=7,title='',bbox_to_anchor=(legendOffset, 0.8),
            labelspacing=.3,
    borderpad=.1,
    handlelength=0,
    columnspacing=2)   

In [ ]:
dfEpiAn.head()

In [ ]:
dfEpiAn.info()

In [ ]:
pltExperiments=['d','e','f']

dfDR=df[idxExclude]
dfEpiAn=dfDR.groupby(['episode','animalIndex','line','group','experiment'],sort=True).mean().reset_index()
dfEpiAn['boutFreq']=1./dfEpiAn.boutDur
dfEpiAn['dishRad']=35
dfEpiAn.loc[(dfEpiAn.experiment=='f'),'dishRad']=50
dfEpiAn['thigNorm']=dfEpiAn.thigmoIndex/dfEpiAn.dishRad

for i,e in enumerate(pltExperiments):
    ix=(dfEpiAn.episode.isin(showepis))&(dfEpiAn.experiment==e)
    data=dfEpiAn[ix]
    plotGroupComparison(data,['ctr','abl'],['si','avgSpeed_smooth','boutFreq','thigNorm'],
                    labels=['Attraction','Swim speed (mm/s)','Bout frequency (Hz)','Thigmotaxis'],
                       ylims=[[0,.5],[0,10],[0,3],[0,1]])
    
    


In [ ]:
ix=(dfEpiAn.episode.isin(showepis))#&(dfEpiAn.experiment==e)
data=dfEpiAn[ix]
dataShoal=data

In [ ]:
lAll=[]
for f in posPath:
    head, tail = os.path.split(f)
    search=head+'\PositionTxt*.txt'
    currTxt=glob.glob(search)[0]
    npyFileOut=currTxt[:-4]+'MapData.npy'
    head, tail = os.path.split(npyFileOut)
    npyFileOut=ProcessingDir+tail
    print('loading:',npyFileOut)
    lAll.append(np.load(npyFileOut))

In [ ]:
allMaps=np.concatenate(tuple(lAll),axis=0)
allMaps.shape

In [ ]:
levels=df['experiment'].unique()
treat=df['group'].unique()[:2]
avgT=np.zeros((len(treat),len(levels),allMaps.shape[3],allMaps.shape[4]))
avgTC=np.zeros((len(treat),len(levels),allMaps.shape[3],allMaps.shape[4]))

In [ ]:
len(levels)

In [ ]:
treat

In [ ]:
levels

In [ ]:
for t,tr in enumerate(treat):
    for i in range(len(levels)):
        ix=np.where((df['episode'].isin(showepis)) &(df['experiment']==levels[i]) & (df['group']==tr)&idxExclude)[0]
        avgT[t,i,:,:]=allMaps[ix,0,0,:,:].mean(axis=0)
        avgTC[t,i,:,:]=allMaps[ix,0,1,:,:].mean(axis=0)

In [ ]:
import skimage
blurred = skimage.filters.gaussian(
    avgT[t,i,:,:], sigma=(3, 3))
plt.imshow(blurred,cmap='gray')

In [ ]:
from mpl_toolkits.axes_grid1 import AxesGrid
import matplotlib
def shiftedColorMap(cmap, start=0, midpoint=0.5, stop=1.0, name='shiftedcmap'):
    '''
    Function to offset the "center" of a colormap. Useful for
    data with a negative min and positive max and you want the
    middle of the colormap's dynamic range to be at zero

    Input
    -----
      cmap : The matplotlib colormap to be altered
      start : Offset from lowest point in the colormap's range.
          Defaults to 0.0 (no lower ofset). Should be between
          0.0 and `midpoint`.
      midpoint : The new center of the colormap. Defaults to 
          0.5 (no shift). Should be between 0.0 and 1.0. In
          general, this should be  1 - vmax/(vmax + abs(vmin))
          For example if your data range from -15.0 to +5.0 and
          you want the center of the colormap at 0.0, `midpoint`
          should be set to  1 - 5/(5 + 15)) or 0.75
      stop : Offset from highets point in the colormap's range.
          Defaults to 1.0 (no upper ofset). Should be between
          `midpoint` and 1.0.
    '''
    cdict = {
        'red': [],
        'green': [],
        'blue': [],
        'alpha': []
    }

    # regular index to compute the colors
    reg_index = np.linspace(start, stop, 257)

    # shifted index to match the data
    shift_index = np.hstack([
        np.linspace(0.0, midpoint, 128, endpoint=False), 
        np.linspace(midpoint, 1.0, 129, endpoint=True)
    ])

    for ri, si in zip(reg_index, shift_index):
        r, g, b, a = cmap(ri)

        cdict['red'].append((si, r, r))
        cdict['green'].append((si, g, g))
        cdict['blue'].append((si, b, b))
        cdict['alpha'].append((si, a, a))

    newcmap = matplotlib.colors.LinearSegmentedColormap(name, cdict)
    plt.register_cmap(cmap=newcmap)

    return newcmap

In [ ]:
treatName=np.array(['Ablation','Control'])
treatNameSimple=np.array(['Abl','Ctr'])

In [ ]:
treat

In [ ]:
levels

In [ ]:
pfp.paper()
inToCm=2.54
trClean=['Ctr','Abl']

for i,lev in enumerate(levels):

    fig, axes = plt.subplots(2,1,figsize=(1, 2),
                            sharex=True, sharey=True)
    axes=axes.ravel()
    #axesCB=[fig.add_subplot() ]
    m=np.nanpercentile(avgT,99.8)
    orig_cmap = matplotlib.cm.coolwarm
    cmap=shiftedColorMap(orig_cmap,midpoint=1-(m/(m+1)))
    trLab=treatName
    pal=['r','.4','.2','k']
    order=np.array([1,0])
    for an,tr in enumerate(treat):

        ind=order[an]
        im = axes[ind].imshow(avgT[an,i,:,:],clim=[0,m],extent=[-31,31,-31,31],origin='lower',cmap=cmap)
        axes[ind].tick_params(axis='y', which='both',length=0)
        axes[ind].tick_params(axis='x', which='both',length=0)
        axes[ind].set_xticks([])
        axes[ind].set_yticks([])
        axes[ind].spines['top'].set_color('white')
        axes[ind].spines['bottom'].set_color('white')
        axes[ind].spines['left'].set_color('white')
        axes[ind].spines['right'].set_color('white')
        axes[ind].set_ylabel(trClean[ind],labelpad=1)
        #axes[ind].text(-30,20,trClean[an])
    print(lev)
    #plt.show()
    plt.subplots_adjust(wspace=0, hspace=0.1)
    
    cax = fig.add_axes([0.9, 0.53, 0.05, 0.34])
    cbar=plt.colorbar(im,cax=cax,ticks=np.round([0,1,5]))
    cbar.ax.tick_params(axis='y', direction='inout',pad=2)
    axes[ind].text(30,-85,'Neighbor\ndensity:',rotation=90)
    #cbar=plt.colorbar(im,ticks=np.round([0,1,5]))

    figPath=FigureDir+'4c_81c_Nmaps_'+lev+'.svg'
    fig.savefig(figPath,bbox_inches='tight')
    
social=df[idxExclude].groupby(['group','episode','animalIndex']).si.mean().reset_index()




In [ ]:
social.head()

In [ ]:

quantRange=[20,40]



In [ ]:
def mapAversion(NeighborMap,nSamp=24,c=(31,31),l=29):
    nSamp=nSamp #number of samples around circle
    cx,cy=c[0],c[1] #center of circle
    l=l #length of intensity profiles outward from center

    samp=np.linspace(0,2*np.pi,nSamp)
    maxAll=np.zeros((nSamp,7))
    
    blurred = skimage.filters.gaussian(
    NeighborMap, sigma=(3, 3))
    
    profileAll=np.zeros((nSamp,l))

    for i,s in enumerate(samp):
        
        a=np.sin(s)*l+cx
        b=np.cos(s)*l+cy
        profile=skimage.measure.profile_line(NeighborMap, (31,31), (a,b), linewidth=5,mode='reflect')
        profileAll[i]=profile[:l]
        r=np.argmax(profile)
        my=np.sin(s)*r+cx
        mx=np.cos(s)*r+cy
        mi=np.nanmax(profile)
        attraction=r*mi
        aversion=attraction-profile[:r].sum()
        aversion=aversion/l
        dip=mi-np.mean(profile[:2])
        maxAll[i]=[mx,my,r,mi,attraction,aversion,dip]
        
    profile=np.mean(profileAll,axis=0)
    r=np.argmax(profile)
    my=np.sin(s)*r+cx
    mx=np.cos(s)*r+cy
    mi=np.nanmax(profile)
    attraction=r*mi
    aversion=attraction-profile[:r].sum()
    aversion=aversion/l
    dip=mi-np.mean(profile[:2])

    maxMean=[mx,my,r,mi,attraction,aversion,dip]
    
    return maxAll,maxMean,profileAll

In [ ]:
col=df.groupby('animalIndex').group.first().values#treatment

In [ ]:
df.anIx.unique()

In [ ]:
levels

In [ ]:
levels=df['experiment'].unique()
Animals=df['animalIndex'].unique()

avgTa=np.zeros((len(Animals),len(levels),allMaps.shape[3],allMaps.shape[4]))
avgTCa=np.zeros((len(Animals),len(levels),allMaps.shape[3],allMaps.shape[4]))
dfPQ=pd.DataFrame()

#fig,ax=plt.subplots(20,7,figsize=(7,20),sharey=True,sharex=True)
#ax=ax.ravel()
j=0
for t,c in enumerate(col):
    for i in range(len(levels)):
        
        ix=np.where((df['episode'].isin(showepis)) &(df['experiment']==levels[i]) & (df['anIx']==t)&idxExclude)[0]
        #print(t,c,col[t])
        avgTa[t,i,:,:]=np.nanmean(allMaps[ix,0,0,:,:],axis=0)
        avgTCa[t,i,:,:]=np.nanmean(allMaps[ix,0,1,:,:],axis=0)

In [ ]:
currentExp='e'#df.iloc[np.where(df['anIx']==t)[0][0]].experiment

i=np.where(levels==currentExp)[0][0]
i

In [ ]:
levels=df['experiment'].unique()
Animals=df['animalIndex'].unique()


dfPQ=pd.DataFrame()

#fig,ax=plt.subplots(20,7,figsize=(7,20),sharey=True,sharex=True)
#ax=ax.ravel()
j=0
for t,c in enumerate(col):
    
    currentExp=df.iloc[np.where(df['anIx']==t)[0][0]].experiment
    
    i=np.where(levels==currentExp)[0][0]
    ix=np.where((df['episode'].isin(showepis)) & (df['anIx']==t)&idxExclude)[0]
    currentMap=np.nanmean(allMaps[ix,0,0,:,:],axis=0)
    #avgTCa[t,i,:,:]=np.nanmean(allMaps[ix,0,1,:,:],axis=0)

    blurred = skimage.filters.gaussian(
    currentMap, sigma=(3, 3))

    #profile=avgTa[t,i,29:31,:].mean(axis=0)
    profile=blurred[quantRange[0]:quantRange[1],:].mean(axis=0)

    profileC=avgTCa[t,i,quantRange[0]:quantRange[1],:].mean(axis=0)
    profileSub=profile/profileC
    mid=profile[quantRange[0]:quantRange[1]].mean()
    high=np.nanpercentile(profile,95)
    mid1=28
    mid2=33
    center=np.nanmean(profile[mid1:mid2])
    ma=mapAversion(avgTa[t,i])
    #aversion=ma[0].mean(axis=0)[5]
    #radius=ma.mean(axis=0)[2]
    #dip2=ma.mean(axis=0)[6]
    aversion=ma[1][5]
    radius=ma[1][2]
    dip2=ma[1][6]
    if j<0:#140:
        ax[t,i].imshow(currentMap,cmap='gray',clim=(0,5))
        profile=np.median(ma[2],axis=0)
        ax[t,i].plot(profile)
        ax[t,i].axhline(high,c='r')
        ax[t,i].axhline(center,c='k')
        ax[t,i].set_title(levels[i]+str(t))
        j=j+1
    dip=high-center
    sub=profileSub[quantRange[0]:quantRange[1]].mean()

    #print t,i,{'an':t,'episode':levels[i],'treat':treatment[t],'lo':lo,'high':high}
    tmp=pd.DataFrame({'an':[t],
                      'episode':levels[i],
                      'treat':col[t],
                      'mid':[mid],
                      'high':[high],
                      'sub':[sub],
                      'center':[center],
                      'dip':[dip],
                      'dip2':[dip2],
                      'radius':[radius],
                     'aversion':[aversion]})
    dfPQ=pd.concat([dfPQ,tmp])
    #print(t,c,i,sum((df['episode']==levels[i]) & (df['anIx']==t)&idx))

In [ ]:
i=1
t=11

ix=np.where((df['episode'].isin(showepis)) &(df['experiment']==levels[i]) & (df['anIx']==t)&idxExclude)[0]
currentMap=np.nanmean(allMaps[ix,0,0,:,:],axis=0)
#avgTCa[t,i,:,:]=np.nanmean(allMaps[ix,0,1,:,:],axis=0)

blurred = skimage.filters.gaussian(
currentMap, sigma=(3, 3))
plt.imshow(blurred)

In [ ]:
col

In [ ]:
ma=mapAversion(avgTa[t,i])

In [ ]:
def normGroup(x):
    return x.mid-x.mid.values[0]

In [ ]:
dfPQ.groupby(['an']).mean()

In [ ]:
dfPQ.episode.unique()

In [ ]:
palSimplePoint=['k','r']
ix=(dfPQ.treat!='e')
sns.pointplot(data=dfPQ[ix],
             x='episode',
             hue='treat',
             y='aversion',
             ci='sd',
              join=False,
             dodge=0.3,
             linestyles=':',
             palette=palSimplePoint)

sns.stripplot(data=dfPQ[ix],
             x='episode',
             hue='treat',
             y='aversion',
             dodge=0.3,
              alpha=0.5,
             linestyles=':',
             palette=palSimplePoint)

In [ ]:
dfPQ.head()

In [ ]:
pfp.paper()
fig, ax = plt.subplots(figsize=(2/inToCm,4.5/inToCm))
palSimpleBar=['r','gray']

ix=(dfPQ.treat!='e')#&(dfPQ.episode.isin(showepis))
#ix=(dfPQ.treat!='dummy')&(dfPQ.episode=='07k01f')
data=dfPQ[ix]


In [ ]:
dataRepulse=data

In [ ]:
data.head()

In [ ]:
treat

In [ ]:
levels=df['experiment'].unique()
treat=df['group'].unique()



fig, axes = plt.subplots(nrows=2, ncols=3, sharey=True,figsize=(5, 5))
m=np.nanpercentile(avgT,95)
#trLab=['c','abl']
for an,tr in enumerate(treat[:2]):
    for i in range(len(levels)):
        profile=avgT[an,i,quantRange[0]:quantRange[1],:].mean(axis=0)
        high=np.nanpercentile(profile,90)
        mid1=28
        mid2=33
        mid=np.nanmean(profile[mid1:mid2])
        axes[an,i].plot(profile)
        axes[an,i].set_title('a:'+str(an)+trLab[an]+ 's:'+levels[i][-4:],fontsize=10)
        axes[an,i].axvline(mid1)
        axes[an,i].axvline(mid2)
        axes[an,i].axhline(high,c='r')
        axes[an,i].axhline(mid,c='k')

In [ ]:
levels

# Loom analysis

In [ ]:
dfTmp=pd.read_csv(posPath[0],
               delim_whitespace=True,
               header=None,
               nrows=5)


In [ ]:
dfTmp


In [ ]:
posPath

In [ ]:
dfTmp1=pd.read_csv(posPath[0],
               delim_whitespace=True,
               header=None,
               nrows=5)

dfTmp=pd.read_csv(posPath[0],
               delim_whitespace=True,
               header=None,
               usecols=[dfTmp1.shape[1]-1],
                  names=['episode'])

CLstart=np.where(dfTmp.episode.str.find('CLfull001L')!=-1)[0]
CLstart

In [ ]:
useCols=np.append(np.arange(2,3*nrAnimalsPerDish,3),nrAnimalsPerDish*3+3)
useCols

In [ ]:
clEpiLen=90*30
nStim=16
clCycle=clEpiLen*nStim
cycles=7
nStimTotal=nStim*cycles

cycles*clCycle/60/30
print('clEpiLen:',clEpiLen)
print('nStimTotal:',nStimTotal)

In [ ]:
colNames=np.append(df.animalIndex.unique(),'episode')
colNames

In [ ]:
posPath

In [ ]:
dfTmp=pd.DataFrame()

#for pp,i in enumerate([0,1,3]):
#for pp,i in enumerate([2,4]):

startAn=0
for i,pp in enumerate(posPath):
    
    nrAnimalsPerDish=info.anPdish.values[i]
    print(pp,nrAnimalsPerDish,' animals per setup')
    
    dfTmp1=pd.read_csv(posPath[i],
               delim_whitespace=True,
               header=None,
               nrows=5)

    dfTmp2=pd.read_csv(posPath[i],
                   delim_whitespace=True,
                   header=None,
                   usecols=[dfTmp1.shape[1]-1],
                      names=['episode'])

    CLstart=np.where(dfTmp2.episode.str.find('CLfull001L')!=-1)[0]
    
    
    
    
    
    indArr=[np.append(np.repeat(df.animalIndex.unique()[startAn:startAn+nrAnimalsPerDish],3),
                      ['StimX','StimY','StimSize','epi']).tolist(),
            np.append(np.tile(['x','y','o'],nrAnimalsPerDish),['a','b','c','d']).tolist()]
    startAn=startAn+nrAnimalsPerDish
    
    tuples = list(zip(*indArr))
    index = pd.MultiIndex.from_tuples(tuples, names=['first', 'second'])
    dfCL=pd.read_csv(posPath[i],
                  #dtype='int8',
               delim_whitespace=True,
               header=None,
               nrows=cycles*clCycle,
               skiprows=CLstart[0],
              names=index,
                 skip_blank_lines=False,
                index_col=np.arange(nrAnimalsPerDish*3,nrAnimalsPerDish*3+4))
    dfCL=dfCL.fillna(0)
    dfCL.index.names = ['StimX','StimY','StimSize','epi']
    dfCL['epFrame']=np.tile(np.arange(clEpiLen),nStimTotal)
    dfCL['epNr']=np.repeat(np.arange(nStimTotal),clEpiLen)
    dfCL.set_index([dfCL.index,'epFrame','epNr'],inplace=True)
    dfCL=dfCL.stack(level=0,
               dropna=False).reset_index()
    dfCL.rename(columns={'level_6':'animalID'}, inplace=True)

    dfCL.sort_values(by=['animalID','epNr','epFrame'],inplace=True)
    dfCL=dfCL.reset_index()

    keepFrames=np.arange(100,250,1)
    print('dfCL.shape',dfCL.shape)
    ix=dfCL.epFrame.isin(keepFrames)
    dfCL=dfCL[ix].reset_index()
    print('dfCL.shape NEW',dfCL.shape)
    dfTmp=pd.concat([dfTmp,dfCL])

In [ ]:
dfCL=dfTmp
dfCL.animalID=dfCL.animalID.astype('int')

In [ ]:
dfCL.animalID.unique()

In [ ]:
dfCL['xDiff']=dfCL.groupby(['animalID','epNr']).x.diff()
dfCL['yDiff']=dfCL.groupby(['animalID','epNr']).y.diff()
dfCL.tail()

In [ ]:
dfCL['dist']=np.sqrt(dfCL.xDiff**2+dfCL.yDiff**2)
dfCL.head()

In [ ]:
plt.hist(dfCL.dist,np.linspace(0,700,30));

plt.yscale('log')
np.sum(dfCL.dist>100)

In [ ]:
dfCL['distClean']=dfCL.dist
dfCL.loc[dfCL.dist>100,'distClean']=0

In [ ]:
plt.plot(dfCL.groupby('epFrame').dist.mean())
plt.plot(dfCL.groupby('epFrame').distClean.mean(),'k')

In [ ]:
#helper functions to convert cartesian coordinates to polar and back. Used to rotate trajectories around origin
def cart2pol(x, y):
    theta = np.arctan2(y, x)
    rho = np.hypot(x, y)
    return theta, rho
def pol2cart(theta, rho):
    x = rho * np.cos(theta)
    y = rho * np.sin(theta)
    return x, y

def rotateTrajectories(df):
    trajectoryLen=150
    dotStart=150
    totalTrials=df.shape[0]/trajectoryLen

    startIndex=df.epFrame==dotStart
    startX=np.repeat(df.x[startIndex].values,trajectoryLen)
    startY=np.repeat(df.y[startIndex].values,trajectoryLen)
    startO=np.repeat(df.o[startIndex].values,trajectoryLen)
    df['xOriginal']=df.x
    df['yOriginal']=df.y

    x=df[['x','StimX']]-np.tile(startX,(2,1)).T
    y=df[['y','StimY']]-np.tile(startY,(2,1)).T
    traT,traR=cart2pol(x.values,y.values)
    traTn=(traT-np.tile(startO,(2,1)).T)+np.tile((np.pi/2.),(2,1)).T
    x,y=pol2cart(traTn,traR)
    
    #t1=(-2*df.l+1).astype(int)
    #t1=np.tile(t1,(2,1)).T
    #x=t1*x
    df[['x','StimX']]=x
    df[['y','StimY']]=y
    
    df['front']=1*(df.y>0) # animal position forwards relative to dot start.
    r,t=cart2pol(df.x.values,df.y.values)
    df['r']=r #r indicates rho - angle from origin to current position.
    df['centerDist']=t #distance from origin.
    return df

In [ ]:
dfCLrot=rotateTrajectories(dfCL)

In [ ]:
dfCLrot.animalID=dfCLrot.animalID.astype('int')

In [ ]:
infoAn.head()

In [ ]:
dfCLrot=pd.merge(dfCLrot,infoAn[['anNr','line','group']],left_on='animalID',right_on='anNr',how='left')
dfCLrot=dfCLrot[~dfCLrot.animalID.isin(excludeAnimals)]
dfCLrot.head()

In [ ]:
dfCLrot.epi.unique()

In [ ]:
dfCLrot

In [ ]:
ix=(dfCLrot.epFrame==175)&(dfCLrot.epi=='CLfull001L')
plt.plot(dfCLrot[ix].groupby(['animalID','epi']).centerDist.mean().unstack(),'.')

In [ ]:
dfCLrot.animalID.unique()

In [ ]:
fig, axes = plt.subplots(3,3,figsize=(10, 10),sharey=True,sharex=True)
axes=axes.ravel()

epiList=dfCLrot.epi.unique()[::2]

for i,e in enumerate(epiList):
    ix=(dfCLrot.epi==e)&(dfCLrot.epFrame<175)&(dfCLrot.epFrame>150)&(dfCLrot.group=='ctr')
    axes[i].plot(dfCLrot[ix].groupby(['epFrame','epi']).centerDist.mean().unstack(),'k-.')
    ix=(dfCLrot.epi==e)&(dfCLrot.epFrame<175)&(dfCLrot.epFrame>150)&(dfCLrot.group=='abl')
    axes[i].plot(dfCLrot[ix].groupby(['epFrame','epi']).centerDist.mean().unstack(),'r-.')
    ix=(dfCLrot.epi==e)&(dfCLrot.epFrame<175)&(dfCLrot.epFrame>150)&(dfCLrot.group=='e')
    axes[i].plot(dfCLrot[ix].groupby(['epFrame','epi']).centerDist.mean().unstack(),'g-.')
    axes[i].set_title(e)


#plt.xlim([150,175])

In [ ]:
fig, axes = plt.subplots(3,3,figsize=(10, 10),sharey=True,sharex=True)
axes=axes.ravel()

epiList=dfCLrot.epi.unique()[::2]

for i,e in enumerate(epiList):
    ix=(dfCLrot.epi==e)&(dfCLrot.epFrame<175)&(dfCLrot.epFrame>150)&(dfCLrot.group=='ctr')
    #axes[i].plot(dfCLrot[ix].groupby(['epFrame','epi']).centerDist.mean().unstack(),'k-.')
    sns.pointplot(data=dfCLrot[ix],x='epFrame',y='centerDist',ax=axes[i],color='k',ci='sd')
    ix=(dfCLrot.epi==e)&(dfCLrot.epFrame<175)&(dfCLrot.epFrame>150)&(dfCLrot.group=='abl')
    sns.pointplot(data=dfCLrot[ix],x='epFrame',y='centerDist',ax=axes[i],color='r',ci='sd')
    #axes[i].plot(dfCLrot[ix].groupby(['epFrame','epi']).centerDist.mean().unstack(),'r-.')
    ix=(dfCLrot.epi==e)&(dfCLrot.epFrame<175)&(dfCLrot.epFrame>150)&(dfCLrot.group=='e')
    sns.pointplot(data=dfCLrot[ix],x='epFrame',y='centerDist',ax=axes[i],color='g',ci='sd')
    axes[i].set_title(e)


#plt.xlim([150,175])

In [ ]:
dfCLrot.animalID=dfCLrot.animalID.astype('int')

In [ ]:
fig, axes = plt.subplots(2,2,figsize=(11, 11),sharey=True,sharex=True)
axes=axes.ravel()

ix=(dfCLrot.epi=='CLfull035R')&(dfCLrot.epFrame<200)&(dfCLrot.epFrame>150)&(dfCLrot.group=='ctr')
axes[0].scatter(dfCLrot[ix].x,dfCLrot[ix].y,s=2,alpha=.3)

ix=(dfCLrot.epi=='CLfull035L')&(dfCLrot.epFrame<200)&(dfCLrot.epFrame>150)&(dfCLrot.group=='ctr')
axes[1].scatter(dfCLrot[ix].x,dfCLrot[ix].y,s=2,alpha=.3)

ix=(dfCLrot.epi=='CLfull035R')&(dfCLrot.epFrame<200)&(dfCLrot.epFrame>150)&(dfCLrot.group=='abl')
axes[2].scatter(dfCLrot[ix].x,dfCLrot[ix].y,s=2,alpha=.3)

ix=(dfCLrot.epi=='CLfull035L')&(dfCLrot.epFrame<200)&(dfCLrot.epFrame>150)&(dfCLrot.group=='abl')
axes[3].scatter(dfCLrot[ix].x,dfCLrot[ix].y,s=2,alpha=.3)

#ix=(dfEarly.l==0)&(dfEarly.treatment==0)
#axes[0].scatter(dfEarly[ix].x,dfEarly[ix].y,s=2)

#ix=(dfEarly.l==1)&(dfEarly.treatment==1)
#axes[1].scatter(dfEarly[ix].x,dfEarly[ix].y,s=2)
#ix=(dfEarly.l==0)&(dfEarly.treatment==1)
#axes[1].scatter(dfEarly[ix].x,dfEarly[ix].y,s=2)

for ax in axes:
    ax.axhline(0,c='k')
    ax.axvline(0,c='k')

In [ ]:
fig, axes = plt.subplots(2,2,figsize=(11, 11),sharey=True,sharex=True)
axes=axes.ravel()

ix=(dfCLrot.epi=='CLfull015R')&(dfCLrot.epFrame<200)&(dfCLrot.epFrame>150)&(dfCLrot.group=='ctr')
axes[0].scatter(dfCLrot[ix].x,dfCLrot[ix].y,s=2,alpha=.3)

ix=(dfCLrot.epi=='CLfull015L')&(dfCLrot.epFrame<200)&(dfCLrot.epFrame>150)&(dfCLrot.group=='ctr')
axes[1].scatter(dfCLrot[ix].x,dfCLrot[ix].y,s=2,alpha=.3)

ix=(dfCLrot.epi=='CLfull015R')&(dfCLrot.epFrame<200)&(dfCLrot.epFrame>150)&(dfCLrot.group=='abl')
axes[2].scatter(dfCLrot[ix].x,dfCLrot[ix].y,s=2,alpha=.3)

ix=(dfCLrot.epi=='CLfull015L')&(dfCLrot.epFrame<200)&(dfCLrot.epFrame>150)&(dfCLrot.group=='abl')
axes[3].scatter(dfCLrot[ix].x,dfCLrot[ix].y,s=2,alpha=.3)

#ix=(dfEarly.l==0)&(dfEarly.treatment==0)
#axes[0].scatter(dfEarly[ix].x,dfEarly[ix].y,s=2)

#ix=(dfEarly.l==1)&(dfEarly.treatment==1)
#axes[1].scatter(dfEarly[ix].x,dfEarly[ix].y,s=2)
#ix=(dfEarly.l==0)&(dfEarly.treatment==1)
#axes[1].scatter(dfEarly[ix].x,dfEarly[ix].y,s=2)

for ax in axes:
    ax.axhline(0,c='k')
    ax.axvline(0,c='k')

## sometimes there are tracking errors in one of the experiment. remove unrealistic centroid jumps...

In [ ]:
fig, axes = plt.subplots(1,2,figsize=(11, 11),sharey=True,sharex=True)

ix=(dfCLrot.group=='ctr')&(dfCLrot.epFrame<200)&(dfCLrot.epFrame>150)
cmap=sns.diverging_palette(240, 10, n=16)
hues=dfCLrot.epi.unique()[np.array([14,12,10,8,6,4,2,0,1,3,5,7,9,11,13,15])]
sns.pointplot(data=dfCLrot[ix],x='epFrame',y='x',hue='epi',ci=None,palette=cmap,hue_order=hues,ax=axes[0])

ix=(dfCLrot.group=='abl')&(dfCLrot.epFrame<200)&(dfCLrot.epFrame>150)

sns.pointplot(data=dfCLrot[ix],x='epFrame',y='x',hue='epi',ci=None,palette=cmap,hue_order=hues,ax=axes[1])

In [ ]:
sfLoomSpeed=dfCL.groupby(['animalID','epi','epFrame','epNr']).distClean.mean().reset_index()
sfLoomSpeed.head()

In [ ]:
fig, axes = plt.subplots(15,10,figsize=(15, 20))
axes=axes.ravel()
for i,a in enumerate(sfLoomSpeed.animalID.unique()):
    ix=(sfLoomSpeed.animalID==a)
    tmp=sfLoomSpeed[ix]
    axes[i].imshow(tmp.pivot_table(index='epFrame',columns='epi',values='distClean').values[50:100].T,
        clim=[0,15],
        cmap='RdBu_r',
        aspect='auto')
    #axes[i].set_title(str(a))



In [ ]:
a=sfLoomSpeed.animalID.unique()[4]

ix=(sfLoomSpeed.animalID==a)
tmp=sfLoomSpeed[ix]
tmp.epi=[x[:-1] for x in tmp.epi]
tmp2=tmp.pivot_table(index='epFrame',columns='epi',values='distClean')
tmp2.plot()

In [ ]:
keepStart=np.array([100,140])
keepLate=-1
ix=(dfCL.epFrame.isin(keepStart))&(dfCL.epNr>keepLate)
dfBL=dfCL[ix]
dfBL['xDiff']=dfBL.groupby(['animalID','epNr']).x.diff()
dfBL['yDiff']=dfBL.groupby(['animalID','epNr']).y.diff()
dfBL['distClean']=np.sqrt(dfBL.xDiff**2+dfBL.yDiff**2)
dfBL=dfBL[dfBL.epFrame==keepStart[1]]

print(dfBL.shape)
dfBL.head()

In [ ]:
keepEnd=np.array([149,180])
keepLate=-1
ix=(dfCL.epFrame.isin(keepEnd))&(dfCL.epNr>keepLate)
dfResp=dfCL[ix]
dfResp['xDiff']=dfResp.groupby(['animalID','epNr']).x.diff()
dfResp['yDiff']=dfResp.groupby(['animalID','epNr']).y.diff()
dfResp['distClean']=np.sqrt(dfResp.xDiff**2+dfResp.yDiff**2)
dfResp=dfResp[dfResp.epFrame==keepEnd[1]]

print(dfResp.shape)
dfResp.head()

In [ ]:
tmp=dfResp.distClean.values/dfBL.distClean.values
tmp[~np.isfinite(tmp)]=0
plt.hist(tmp,bins=np.linspace(0,5,20))

In [ ]:
dfResp['escape']=tmp>2
dfResp['epiSimple']=[x[:-1] for x in dfResp.epi]
dfResp.head()

In [ ]:
dfResp.animalID=dfResp.animalID.astype('int')
dfResp=pd.merge(dfResp,infoAn[['anNr','line','group']],left_on='animalID',right_on='anNr',how='left')
dfResp.head()

In [ ]:
dfResp.sort_values(by='epiSimple',inplace=True)
dfResp['size']=[int(x[-3:-1])/(1280/600) for x in dfResp.epi]

dfResp.tail()

In [ ]:
dfResp.line.unique()

In [ ]:
fig, axes = plt.subplots(figsize=(3, 3))
sns.set_palette('viridis',1)

ix=dfResp.group.isin(['abl','ctr'])
data=dfResp[ix].groupby(['animalID','epiSimple','group']).escape.mean().reset_index()

sns.pointplot(data=data,
              x='epiSimple',
              y='escape',
              hue='group',
              estimator=np.nanmean,
             dodge=True,
             ci='sd')

#plt.xticks(rotation=90);
plt.ylabel('Escape fraction')
plt.xlabel('Loom size (units?)')
loomSizes=np.array([0,5,10,15,20,25,30,35])
axes.set_xticklabels(loomSizes)
plt.legend(frameon=False)
#plt.ylim([-.02,.79])
sns.despine()

In [ ]:
plt.rcParams['svg.fonttype'] = 'none'
fig,axes = plt.subplots(figsize=(3/inToCm,4/inToCm))
sns.set_palette('viridis',1)

sns.lineplot(data=dfResp[(dfResp.line=='81cNTR')&(dfResp.group.isin(['abl','ctr']))].groupby(['animalID','size','group']).escape.mean().reset_index(),
              x='size',
              y='escape',
              hue='group',
             style='group',
             hue_order=['ctr','abl'],
             palette=['k','r'],
              estimator=np.nanmean,
             #ci='sd',
            markers=['.','.'],
            #err_style='bars',
             #alpha=0.5
            )

#plt.xticks(rotation=90);
plt.ylabel('Escape fraction')
plt.xlabel('Loom size (mm)')
#loomSizes=[0,5,10,15,20,25,30,35]
#axes.set_xticklabels(loomSizes)
sns.despine()

handles, labels = axes.get_legend_handles_labels()
axes.legend(handles=handles[0:], labels=['Ctr','Abl'],frameon=False)
plt.title('Looming disc')
figPath=FigureDir+'5_MTZ_Loom_escapeFraction.svg'
plt.savefig(figPath,bbox_inches='tight')

In [ ]:
plt.plot(loomSizes,2*np.rad2deg(np.arctan((loomSizes/4)/20)),'r.-',label='20mm dist')
plt.plot(loomSizes,2*np.rad2deg(np.arctan((loomSizes/4)/5)),'k.-',label='5mm dist')
plt.legend()
plt.ylabel('visual angle (degrees)')
plt.xlabel('loom disc diameter (px)')

In [ ]:
dfResp.animalID.unique()

In [ ]:

ix=(dfResp.group.isin(['abl','ctr']))&(dfResp.epiSimple=="CLfull035")&(~dfResp.animalID.isin(excludeAnimals))
data=dfResp[ix].groupby(['animalID','group']).escape.mean().reset_index()



In [ ]:
dataEscape=data

In [ ]:
print(dataShoal.shape)
print(dataEscape.shape)
print(dataRepulse.shape)

In [ ]:
dataAll=pd.merge(dataShoal,dataRepulse[['aversion','an']],left_on='anIx',right_on='an',how='left')
dataAll=pd.merge(dataAll,dataEscape[['escape','animalID']],left_on='animalIndex',right_on='animalID')
dataAll

In [ ]:
sns.pairplot(data=dataAll,
             hue='group',
             markers="+",
             height=1.5,
             vars=['si','avgSpeed_smooth','thigmoIndex','boutDur','aversion','escape']);

In [ ]:
dataAll.head()

In [ ]:
pltExperiments=['d','e','f']

for i,e in enumerate(pltExperiments):
    ix=(dataAll.episode.isin(showepis))&(dataAll.experiment==e)
    dataX=dataAll[ix]
    plotGroupComparison(dataX,['ctr','abl'],['si','aversion','avgSpeed_smooth','boutFreq','thigNorm','escape'],
                    labels=['Attraction',
                            'Short range \nrepulsion (a.u)',
                            'Swim speed (mm/s)',
                            'Bout frequency (Hz)',
                            'Thigmotaxis',
                            
                           'Escape fraction'],
                       ylims=[[0,.6],[0,.65],[0,10],[0,3],[0,1],[0,1]])

In [ ]:
dataAll.info()

In [ ]:
dataAll[['animalIndex','line','group','si','aversion','avgSpeed_smooth','boutFreq','thigNorm','escape']].to_csv(FigureDir+'f4.csv')
FigureDir+'f4.csv'

In [ ]:
srcDat=dataAll[['animalIndex','line','group','si','aversion','avgSpeed_smooth','boutFreq','thigNorm','escape']]
srcDat

In [ ]:
srcDat.groupby(['line','group']).si.count()

In [ ]:
pltExperiments=['d','e','f']

for i,e in enumerate(pltExperiments):
    ix=(dataAll.episode.isin(showepis))&(dataAll.experiment==e)
    dataX=dataAll[ix]
    plotGroupComparison(dataX,['ctr','abl'],['si','aversion'],
                    labels=['Attraction',
                            'Short range \nrepulsion (a.u)'],
                       ylims=[[0,.6],[0,.60]],
                       legendOffset=2.2)
    figPath=FigureDir+'f4_ablationBeh_'+str(i)+'.svg'
    plt.savefig(figPath,bbox_inches='tight')

In [ ]:
pltExperiments=['d','e','f']

for i,e in enumerate(pltExperiments):
    ix=(dataAll.episode.isin(showepis))&(dataAll.experiment==e)
    dataX=dataAll[ix]
    plotGroupComparison(dataX,['ctr','abl'],['avgSpeed_smooth','boutFreq','thigNorm','escape'],
                    labels=[
                            'Swim speed (mm/s)',
                            'Bout frequency (Hz)',
                            'Thigmotaxis',
                            
                           'Escape fraction'],
                       ylims=[[0,10],[0,3],[0,1],[0,1]],
                       legendOffset=1.8)
    figPath=FigureDir+'f4_ablationCtrBeh_'+str(i)+'.svg'
    plt.savefig(figPath,bbox_inches='tight')

In [ ]:
dataAll.head()

In [ ]:
dataAll.groupby(['experiment','group']).animalIndex.count()

In [ ]:
dataAll

In [ ]:
df1020=dataAll[dataAll.experiment=='e']
df1020.head()

In [ ]:
df1020['groupFine']=[x[-1] for x in df1020.line]
df1020.head()

In [ ]:
pfp.paper()
fig, ax = plt.subplots(figsize=(2.5/inToCm,5/inToCm))

treatName=np.array(['RFP-MTZ-','RFP-MTZ+','RFP+MTZ-','RFP+MTZ+'])
pal=['r','.6','.4','k']

pal=np.array(pal)
si=df1020.groupby(['groupFine','animalIndex']).si.mean().reset_index()
#treatName=np.array(['CTR+MTZ','C81+MTZ'])
#si['treatName']=treatName[speed.group.values.astype('int')]
sns.pointplot(y="si", 
            x='groupFine',
              hue='groupFine',
              #order=treatName[[3,2,1,0]],
              #hue_order=treatName[[3,2,1,0]],
            data=si,
            ax=ax,
              #color='k',
              zorder=100,
              join=False,
              scale=.7,
            palette=pal[[3,2,1,0]],
            linewidth=2,
            ci='sd')

sns.stripplot(y="si", 
            x='groupFine',
            data=si,
            ax=ax,
            palette=pal[[3,2,1,0]],
              #order=treatName[[3,2,1,0]],
              edgecolor='w',
            alpha=.5,
              linewidth=1,
              zorder=-100
            )




plt.xlabel('')
plt.xticks([])
plt.ylabel('Attraction')

l=ax.legend()
h,l = ax.get_legend_handles_labels()
ax.legend(h,treatName,frameon=False,ncol=1,
         loc=1,bbox_to_anchor=(1.7, 1),title='',
                        labelspacing=0,
               borderpad=0,
               handlelength=0,
               columnspacing=2,)   # <<<<<<<< This is where the magic happens

plt.ylim([-.1,.6])
ax.axhline(0,ls=':',color='k')
sns.despine()
plt.subplots_adjust(wspace=0, hspace=0)

figPath=FigureDir+'S7_1026ControlSplit.svg'
plt.savefig(figPath,bbox_inches='tight')

In [ ]:
FigureDir